In [1]:
import os
from google import genai 
from google.genai import types
from dotenv import load_dotenv
import json
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
load_dotenv()

/Users/anus/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/anus/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
def get_chatbot_response(client, model_name, messages):
    input_messages = []
    for message in messages:
        input_messages.append(types.Content(role=message['role'], parts=[types.Part.from_text(text=message['content'])]))
    response = client.models.generate_content(
        model=model_name,
        contents=input_messages,
    #     config = types.GenerateContentConfig(
    #     response_mime_type='application/json',
    #     response_schema=responseFormatType,
    # ),
    ).candidates[0].content.parts[0].text
    return response

# Get LLM response

In [3]:
GEMINI_KEY = os.getenv('GEMINI_API_KEY')
client = genai.Client(api_key=GEMINI_KEY)



messages = [{"role":"user","content":"Whats the capital of Germany?"}]
response = get_chatbot_response(client,"gemini-2.5-flash",messages)

response



'The capital of Germany is **Berlin**.'

# Prompt Engineering

## Structured Output

In [4]:
system_prompt = """
You are a helpful assistant that answer questions about capitals of countries.

Your output should be in a structured json format exactly like the one below. You are not allowed to write anuything other than json object:
[
{
   "country": the country that you will get the capital of
   "capital": the capital of the country stated
}

]
"""

from pydantic import BaseModel

# Define the desired output structure using Pydantic
class StructuredOutput(BaseModel):
    country: str
    capital: str
   #  ingredients: list[str]
   #  steps: list[str]



messages = [{"role":"user", "content":system_prompt}]
messages.append({"role":"user", "content":"Whats the capital of Germany?"})
# messages = [
#     {"role": "user", "content": "Whats the capital of Germany?"}
# ]
response = get_chatbot_response(client,"gemini-2.5-flash",messages)

def cleaned_response(response):
    cleaned = response.strip()

    if cleaned.startswith("```json"):
        cleaned = cleaned.removeprefix("```json").strip()

    if cleaned.endswith("```"):
        cleaned = cleaned.removesuffix("```").strip()
    return cleaned 

processed_response = cleaned_response(response)
print(processed_response)

[
  {
    "country": "Germany",
    "capital": "Berlin"
  }
]


In [5]:
type(processed_response)

str

In [6]:
json_response = json.loads(processed_response)
json_response

[{'country': 'Germany', 'capital': 'Berlin'}]

In [7]:
type(json_response[0]),json_response[0]['capital']

(dict, 'Berlin')

# Input Structuring

In [8]:
user_input = """
Get me the capitals of the following countries:
```
1. Italy
2. Germany
3. France

```
"""

messages = [{"role":"user","content":system_prompt}]
messages.append({"role":"user","content":user_input})
raw_response = get_chatbot_response(client,"gemini-2.5-flash",messages)
response = cleaned_response(raw_response)
print(response)


[
  {
    "country": "Italy",
    "capital": "Rome"
  },
  {
    "country": "Germany",
    "capital": "Berlin"
  },
  {
    "country": "France",
    "capital": "Paris"
  }
]


In [9]:
json_response = json.loads(response)
json_response

[{'country': 'Italy', 'capital': 'Rome'},
 {'country': 'Germany', 'capital': 'Berlin'},
 {'country': 'France', 'capital': 'Paris'}]

# Give the model time to think (Chain of Thought)

In [10]:
user_prompt = """
Calculate the result of the equation: 1+3

Your output should be in a structured json format exactly like the one below. You are not allowed to write anything other than the json object:
{
result: The final number resulted from calculating the equation above
}
"""

messages = [{"role":"user","content":user_prompt}]
raw_response = get_chatbot_response(client,"gemini-2.5-flash", messages)
response = cleaned_response(raw_response)
print(response)


{
"result": 4
}


In [11]:
259/2*8654+91072*33-12971

4113098.0

In [12]:
# Normal direct calculation
user_prompt = """
Calculate the result of the equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one below. You are not allowed to write anything other than the json object:
{
result: The final number resulted from calculating the equation above
}
"""

messages = [{"role":"user","content":user_prompt}]
raw_response = get_chatbot_response(client,"gemini-2.5-flash", messages)
response = cleaned_response(raw_response)
print(response)

{
"result": 4113168
}


In [13]:
# Step by step Calculation

user_prompt = """
Calculate the result of the equation: 259/2*8654+91072*33-12971

Your output should be in a structured json format exactly like the one below. You are not allowed to write anything other than the json object:
{
steps:This is where you solve the equation bit by bit following the BEDMAS order of operations. You need to show your work and calculate each step leading to the final answer. Feel free to write in free text
result: The final number resulted from calculating the equation above
}
"""

messages = [{"role":"user","content":user_prompt}]
raw_response = get_chatbot_response(client,"gemini-2.5-flash", messages)
response = cleaned_response(raw_response)
print(response)

{
  "steps": "We need to calculate the equation 259/2*8654+91072*33-12971 following the BEDMAS order of operations.\n\n1.  **Division:** First, perform the division: 259 / 2 = 129.5\n    The equation becomes: 129.5 * 8654 + 91072 * 33 - 12971\n\n2.  **Multiplication (from left to right):**\n    a. Perform the first multiplication: 129.5 * 8654 = 1120963\n       The equation becomes: 1120963 + 91072 * 33 - 12971\n    b. Perform the second multiplication: 91072 * 33 = 3005376\n       The equation becomes: 1120963 + 3005376 - 12971\n\n3.  **Addition:** Perform the addition: 1120963 + 3005376 = 4126339\n    The equation becomes: 4126339 - 12971\n\n4.  **Subtraction:** Perform the subtraction: 4126339 - 12971 = 4113368",
  "result": 4113368
}


In [14]:
4101368 - 4113098.0

-11730.0

# RAG - Retrival Augmented Generation

In [17]:
user_prompt = """
Whats new in iphone 16?
"""

messages = [{"role":"user", "content":user_prompt}]
raw_response = get_chatbot_response(client,"gemini-2.5-flash", messages)
response = cleaned_response(raw_response)
print(response)

ClientError: 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. \n* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 5, model: gemini-2.5-flash\nPlease retry in 17.256448337s.', 'status': 'RESOURCE_EXHAUSTED', 'details': [{'@type': 'type.googleapis.com/google.rpc.Help', 'links': [{'description': 'Learn more about Gemini API quotas', 'url': 'https://ai.google.dev/gemini-api/docs/rate-limits'}]}, {'@type': 'type.googleapis.com/google.rpc.QuotaFailure', 'violations': [{'quotaMetric': 'generativelanguage.googleapis.com/generate_content_free_tier_requests', 'quotaId': 'GenerateRequestsPerMinutePerProjectPerModel-FreeTier', 'quotaDimensions': {'location': 'global', 'model': 'gemini-

### Giving Context to unknown structure

In [18]:
iphone_16 = """
The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learning to separate background sounds from speech, allowing for more refined audio capture during video recording. Battery life has been extended, especially in the iPhone 16 Pro Max, which is claimed to have the longest-lasting battery of any iPhone 
9TO5MAC

APPLEMAGAZINE
.

Additionally, Apple has switched to USB-C for faster charging and data transfer, and the Pro models now support up to 2x faster video encoding. The starting prices remain consistent with previous generations, with the iPhone 16 starting at $799, while the Pro models start at $999
"""

In [19]:
user_prompt = f"""
{iphone_16}

Whats new in iphone 16?
"""

messages = [{"role":"user", "content":user_prompt}]
raw_response = get_chatbot_response(client,"gemini-2.5-flash", messages)
response = cleaned_response(raw_response)
print(response)

Based on the provided text, here's what's new in the iPhone 16:

*   **Displays:**
    *   Larger displays: 6.1-inch (base model), 6.7-inch (iPhone 16 Plus), 6.3-inch (iPhone 16 Pro), and 6.9-inch (iPhone 16 Pro Max).
    *   Thinner bezels across all models, with Pro models boasting the thinnest bezels on any Apple product.
    *   More durable Ceramic Shield.
*   **Performance:**
    *   New A18 chip (A18 Pro for Pro models).
    *   Significant performance improvements, including enhanced neural engine capabilities, faster GPU for gaming, and improved machine learning tasks.
*   **Cameras:**
    *   Upgraded camera systems.
    *   Base iPhone 16 features a dual-camera setup with a 48MP main sensor.
    *   Pro models offer a 48MP Ultra Wide and 5x telephoto camera.
    *   New "Camera Control" button on Pro models for more flexible photography options.
*   **Audio Features:**
    *   "Audio Mix" feature, which uses machine learning to separate background sounds from speech for refi

In [20]:
# Load model (downloads automatically on first run)
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [21]:
# Generate embeddings
embeddings = model.encode(user_prompt)

### Automatically Extract context data from database

In [22]:
samsung_s23 = """
The Samsung Galaxy S23 brings some incremental but notable upgrades to its predecessor, the Galaxy S22. It features the Snapdragon 8 Gen 2 processor, a powerful chip optimized for the S23 series, delivering enhanced performance, especially for gaming and multitasking. This chip ensures top-tier speed and efficiency across all models, from the base S23 to the larger S23+ and S23 Ultra​
STUFF

TECHRADAR
.

In terms of design, the S23's camera module has been streamlined by removing the raised metal contour around the cameras, creating a cleaner, sleeker look. It also sports the same 6.1-inch 120Hz AMOLED display, protected by tougher Gorilla Glass Victus 2, making it more resistant to scratches and drops​
TECHRADAR
.

The S23 Ultra stands out with its 200MP main camera, offering impressive photo clarity, especially in low-light conditions. The selfie camera across the series has been updated to a 12MP sensor, resulting in sharper selfies. The Ultra model also includes productivity tools such as the S-Pen, which remains an essential feature for note-taking and creative tasks​
STUFF

TECHRADAR
.

Battery life is solid, with the S23 Ultra featuring a 5000mAh battery that lasts comfortably through a day of heavy use. However, charging speeds still lag behind some competitors, with 45W wired charging, which is slower than other brands offering up to 125W charging​
STUFF
.

Overall, the Galaxy S23 series enhances performance, durability, and camera quality, making it a strong contender for users seeking a high-performance flagship.
"""

In [34]:
data = [iphone_16,samsung_s23]

# Generate embeddings
def get_embeddings(text_input):
    output = model.encode(text_input)

    embeddings = []
    for embeddings_object in output:
        embeddings.append(embeddings_object)

    return embeddings
    

In [35]:
user_prompt_embedding = get_embeddings(user_prompt)
user_prompt_embedding


[np.float32(-0.01734082),
 np.float32(0.086646594),
 np.float32(0.02216926),
 np.float32(-0.06114932),
 np.float32(0.03321037),
 np.float32(-0.03662445),
 np.float32(0.02238598),
 np.float32(0.09825371),
 np.float32(0.0013984273),
 np.float32(0.056673568),
 np.float32(0.057190165),
 np.float32(-0.014590337),
 np.float32(-0.05014798),
 np.float32(0.081399105),
 np.float32(0.009564458),
 np.float32(0.059007462),
 np.float32(0.17079087),
 np.float32(-0.051716622),
 np.float32(-0.09448234),
 np.float32(-0.049148925),
 np.float32(0.009755719),
 np.float32(-0.080376394),
 np.float32(0.04315882),
 np.float32(0.020698557),
 np.float32(0.07560777),
 np.float32(0.027003864),
 np.float32(-0.054305498),
 np.float32(-0.03209814),
 np.float32(-0.043194372),
 np.float32(0.01625959),
 np.float32(0.023613919),
 np.float32(0.026111446),
 np.float32(0.11546241),
 np.float32(0.010868181),
 np.float32(-0.03132772),
 np.float32(-0.113708906),
 np.float32(0.049629793),
 np.float32(-0.012059769),
 np.float32(

In [36]:
data_embeddings = [get_embeddings(text) for text in data]
data_embeddings

[[np.float32(-0.01734082),
  np.float32(0.086646594),
  np.float32(0.02216926),
  np.float32(-0.06114932),
  np.float32(0.03321037),
  np.float32(-0.03662445),
  np.float32(0.02238598),
  np.float32(0.09825371),
  np.float32(0.0013984273),
  np.float32(0.056673568),
  np.float32(0.057190165),
  np.float32(-0.014590337),
  np.float32(-0.05014798),
  np.float32(0.081399105),
  np.float32(0.009564458),
  np.float32(0.059007462),
  np.float32(0.17079087),
  np.float32(-0.051716622),
  np.float32(-0.09448234),
  np.float32(-0.049148925),
  np.float32(0.009755719),
  np.float32(-0.080376394),
  np.float32(0.04315882),
  np.float32(0.020698557),
  np.float32(0.07560777),
  np.float32(0.027003864),
  np.float32(-0.054305498),
  np.float32(-0.03209814),
  np.float32(-0.043194372),
  np.float32(0.01625959),
  np.float32(0.023613919),
  np.float32(0.026111446),
  np.float32(0.11546241),
  np.float32(0.010868181),
  np.float32(-0.03132772),
  np.float32(-0.113708906),
  np.float32(0.049629793),
  

In [37]:
len(data_embeddings)

2

In [38]:
data_similarity = cosine_similarity([user_prompt_embedding],data_embeddings)

In [41]:
data_similarity
# The higher the number the higher the similarity

array([[0.9999999, 0.5553126]], dtype=float32)

In [ ]:
# argmax gives the index of high similarity
closest_entry_index= data_similarity.argmax()
closest_entry_index

np.int64(0)

In [45]:
data[closest_entry_index]

'\nThe iPhone 16 introduces several exciting updates, making it one of Apple\'s most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.\n\nPowered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.\n\nApple also introduced advanced audio features like "Audio Mix," which uses machine l

In [46]:
user_prompt_with_data = f"""
{data[closest_entry_index]}

{user_prompt}
"""

In [47]:
print(user_prompt_with_data)



The iPhone 16 introduces several exciting updates, making it one of Apple's most advanced smartphones to date. It features a larger 6.1-inch display for the base model and a 6.7-inch screen for the iPhone 16 Plus, with thinner bezels and a more durable Ceramic Shield. The iPhone 16 Pro and Pro Max boast even larger displays, measuring 6.3 and 6.9 inches respectively, offering the thinnest bezels seen on any Apple product so far.

Powered by the new A18 chip (A18 Pro for the Pro models), these phones deliver significant performance improvements, with enhanced neural engine capabilities, faster GPU for gaming, and machine learning tasks. The camera systems are also upgraded, with the base iPhone 16 sporting a dual-camera setup with a 48MP main sensor. The Pro models offer a 48MP Ultra Wide and 5x telephoto camera, enhanced by Apple’s "Camera Control" button for more flexible photography options.

Apple also introduced advanced audio features like "Audio Mix," which uses machine learnin

In [49]:
messages = [{"role":'user','content':user_prompt_with_data}]
raw_response = get_chatbot_response(client,"gemini-2.5-flash", messages)
response = cleaned_response(raw_response)
print(response)

Based on the provided text, here's what's new in the iPhone 16:

*   **Larger Displays:**
    *   Base model: 6.1-inch
    *   iPhone 16 Plus: 6.7-inch
    *   iPhone 16 Pro: 6.3-inch
    *   iPhone 16 Pro Max: 6.9-inch
*   **Thinner Bezels:** The Pro models feature the thinnest bezels seen on any Apple product so far.
*   **More Durable Ceramic Shield:** Enhanced durability for the display.
*   **New A18 Chip:**
    *   A18 chip for base and Plus models.
    *   A18 Pro chip for Pro models.
    *   Significant performance improvements, enhanced neural engine, and faster GPU.
*   **Upgraded Camera Systems:**
    *   Base iPhone 16: Dual-camera setup with a 48MP main sensor.
    *   Pro models: 48MP Ultra Wide and 5x telephoto camera.
    *   New "Camera Control" button for more flexible photography.
*   **Advanced Audio Features:**
    *   "Audio Mix": Uses machine learning to separate background sounds from speech for refined audio capture during video recording.
*   **Extended Batter